# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 단순 질의용 prompt 생성-from_template() 메소드 이용. (PromptTemplate용)
prompt = ChatPromptTemplate.from_template(
    template="한국의 {topic}에 관련된 속담 {count}개를 알려줘. 목록 형식으로 출력해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()
# prompt -> model -> parser

query = prompt.invoke({"topic":"호랑이", "count":3})
res = model.invoke(query)
final_res = parser.invoke(res)

In [ ]:
res

AIMessage(content='1. 호랑이도 제 말하면 온다 — 어떤 사람을 이야기하면 그 사람이 실제로 나타날 때 쓰는 말(영어의 "speak of the devil"와 유사).  \n2. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다 — 큰 성과를 얻으려면 위험을 감수해야 한다는 뜻.  \n3. 범에게 물려가도 정신만 차리면 산다 — 위험한 상황에서도 침착하면 위기를 벗어날 수 있다는 뜻.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 955, 'prompt_tokens': 30, 'total_tokens': 985, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnbHElKxhzQzw9gZ2y08COPRCPuWW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a12-4586-7bd0-b308-76bf95c73de5-0', usage_metadata={'input_tokens': 30, 'output_tokens': 955, 'total_tokens': 985, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

In [6]:
print(final_res)

1. 호랑이도 제 말하면 온다 — 어떤 사람을 이야기하면 그 사람이 실제로 나타날 때 쓰는 말(영어의 "speak of the devil"와 유사).  
2. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다 — 큰 성과를 얻으려면 위험을 감수해야 한다는 뜻.  
3. 범에게 물려가도 정신만 차리면 산다 — 위험한 상황에서도 침착하면 위기를 벗어날 수 있다는 뜻.


## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [10]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
txt = "이순신,유관순,안중근,강감찬,세종대왕"
txt = "요청하신 이름은 이순신과 유관순과 안중근입니다."
r1 = parser.parse(txt)
r2 = parser.invoke(txt)
print(type(r1), type(r2))

<class 'list'> <class 'list'>


In [11]:
print(r1)
print(r2)

['요청하신 이름은 이순신과 유관순과 안중근입니다.']
['요청하신 이름은 이순신과 유관순과 안중근입니다.']


In [ ]:
# 이 output parser에 맞는 출력을 모델이 하도록 하는 지시문으로 프롬프트에 추가한다.
parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [27]:
parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate(
    messages=[
        {"role":"system", "content": "Output Format: {format_instruction}"},
        {"role":"user", "content":"{subject}에 대해 다섯가지를 나열해 주세요."}
    ],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
# partial_variables={"input_variable":넣을 값,} => input_variable의 값을 Prompt Template을 
#   만들면서 넣어준다. (invoke 시 넣는 것이 아니라 생성시 설정.)
model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"subject":"자동차 종류"})
res = model.invoke(query)

In [28]:
print(res.content)
final_res = parser.invoke(res)
final_res

세단, SUV, 해치백, 쿠페, 컨버터블


['세단', 'SUV', '해치백', '쿠페', '컨버터블']

In [29]:
# 질의 ->(Prompt)->query->(Model)->res->(Parser)->최종답변
# Chain: 위 실행흐름을 자동화 (pipeline)
chain = prompt | model | parser
res = chain.invoke({"subject":"오픈소스 LLM 모델 이름"})
print(res)

['Llama 2', 'Falcon 40B', 'MPT-7B', 'BLOOM', 'GPT-NeoX-20B']


## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [38]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
print(parser.get_format_instructions())

txt = '{"name":"홍길동","age":20}'
# txt = "이름: 홍길동, 나이: 20"
r = parser.invoke(txt)
print(type(r))
r['age']

Return a JSON object.
<class 'dict'>


20

In [ ]:
# JSON 응답 스키마(구조 설계) 정의 -> Pydantic
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")

# 변수: key 
# Field.description: 변수에 넣을 내용

In [59]:
parser2 = JsonOutputParser(pydantic_object=PersonInfo)
print(parser2.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [60]:
# parser = JsonOutputParser()
parser = JsonOutputParser(pydantic_object=PersonInfo)
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [61]:
final_res = parser.invoke(res)

In [ ]:
print(type(res.content), type(final_res))
final_res

<class 'str'> <class 'dict'>


{'name': '이순신',
 'yob': 1545,
 'yod': 1598,
 'profile': '조선 중기의 무신이자 명장. 임진왜란(1592-1598) 동안 조선 수군을 지휘하여 다수의 해전에서 승리했고, 거북선과 학익진 전술을 활용해 해상 우위를 확보했다. 한산도 대첩, 명량 해전(13척으로 다수의 적을 격파), 노량 해전에서 전사(1598) 등으로 조선을 지키는 데 결정적 기여를 했으며, 충무공이라는 시호로 추앙받는다.'}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [65]:
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")

In [66]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=PersonInfo)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "조회한 사람의 이름", "title": "Name", "type": "string"}, "yob": {"description": "조회한 사람이 태어난 년도", "title": "Yob", "type": "integer"}, "yod": {"description": "조회한 사람이 사망한 년도", "title": "Yod", "type": "integer"}, "profile": {"description": "조회한 사람의 주요 업적", "title": "Profile", "type": "string"}}, "required": ["name", "yob", "yod", "profile"]}
```


In [67]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=PersonInfo)

prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [ ]:
res

AIMessage(content='{"name":"이순신","yob":1545,"yod":1598,"profile":"조선 중기의 해군 장군. 임진왜란(1592–1598) 동안 탁월한 지휘로 한산도 대첩(1592)·명량 해전(1597) 등에서 결정적 승리를 거두었으며 거북선과 학익진 등 전술을 활용해 일본 수군을 격파했다. 난중일기 등 기록을 남겼고, 1598년 노량해전에서 전사했으며 사후 충무공으로 추서되었다."}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 721, 'prompt_tokens': 274, 'total_tokens': 995, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cnccw7En0WJJYl1VHvrjWXifGwJf7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a61-7a9a-7910-92ac-4d7708e91a2c-0', usage_metadata={'input_tokens': 274, 'output_tokens': 721, 'total_tokens': 995, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [69]:
final_res = parser.invoke(res)
type(final_res)

__main__.PersonInfo

In [70]:
final_res

PersonInfo(name='이순신', yob=1545, yod=1598, profile='조선 중기의 해군 장군. 임진왜란(1592–1598) 동안 탁월한 지휘로 한산도 대첩(1592)·명량 해전(1597) 등에서 결정적 승리를 거두었으며 거북선과 학익진 등 전술을 활용해 일본 수군을 격파했다. 난중일기 등 기록을 남겼고, 1598년 노량해전에서 전사했으며 사후 충무공으로 추서되었다.')

In [72]:
final_res.name
final_res.profile

'조선 중기의 해군 장군. 임진왜란(1592–1598) 동안 탁월한 지휘로 한산도 대첩(1592)·명량 해전(1597) 등에서 결정적 승리를 거두었으며 거북선과 학익진 등 전술을 활용해 일본 수군을 격파했다. 난중일기 등 기록을 남겼고, 1598년 노량해전에서 전사했으며 사후 충무공으로 추서되었다.'

# LLM모델에 출력 형식을 설정

- ChatModel객체의 `with_structured_output(pydantic.BaseModel)` 을 이용해 모델의 출력 형식을 모델 자체에 추가할 수있다.
- `OutputParser`는 모델의 출력 결과를 받아서 형식을 변경해 준다. 그래서 Chain에 탈/부착을 통해 형식을 적용하거나 적용하지 않는 것을 자유롭게 할 수있다.
- 모델의 출력 결과를 항상 일정하게 할 경우에는 아예 **모델에 출력 형식을 설정할 수 있다.**

In [73]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# 출력 schema 정의
class OutputSchema(BaseModel):
    is_real_person: bool = Field(..., 
        description="요청한 인물이 실존인물인지 여부. True: 실존인물, False: 실존인물이 아님")
    description: str = Field(..., description="요청한 인물에 대한 설명")

model = ChatOpenAI(model='gpt-5-mini')
res = model.invoke("이순신 장군에 대해 설명해줘.")

In [74]:
print(type(res))
print(res.content)

<class 'langchain_core.messages.ai.AIMessage'>
아래는 이순신(李舜臣) 장군에 대한 간단하고 핵심적인 설명입니다.

- 기본 정보
  - 생몰: 1545년 ~ 1598년. 조선 중기 무신이자 해군 장수.
  - 시호·칭호: 충무공(忠武公) 등으로 불리며 한국의 대표적 국난극복 영웅이다.

- 활동 배경
  - 임진왜란(1592–1598) 당시 일본의 침략에 맞서 조선 수군을 지휘해 해상에서 연이어 대승을 거뒀다.
  - 해상 교통로와 보급로를 장악함으로써 육지 전투에 큰 기여를 했다.

- 주요 전과와 전술
  - 한산도 대첩(1592) 등 여러 해전에서 큰 승리를 거두며 일본 수군의 기동을 봉쇄했다.
  - 명량 해전(1597, 명량대첩)에서 12척 또는 13척의 배로 수십~수백 척의 일본 함대를 상대로 대승을 거둔 것은 그의 최고 전술적 업적으로 꼽힌다.
  - 그 외 옥포, 당포, 부산포 등 여러 해전에서 승리했다.
  - “학익진”(학의 날개 모양 전진 배치) 같은 기동·집중 공격 전술과 해협의 지형을 활용한 전술 운용이 특징이다.
  - 전투 기록에 따르면 큰 해전에서 패배 없이 여러 차례 승리를 거두었고, 종종 “23전 23승, 전선 단 한 척도 잃지 않았다”는 평가를 받는다(다만 숫자 표기는 자료마다 차이가 있다).

- 거북선(거북선)과 함선 운용
  - 거북선(거북선)은 장갑으로 덮인 전열함 형태로 전설적 상징이 되었으나, 실제 구조·기원에 대해서는 학자들 사이에 논쟁이 있다. 다만 이순신이 판옥선 위주로 강력한 화력을 집중·운용하고, 신무기와 전술적 혁신을 도입해 해상 우위를 확보했다는 점은 확실하다.

- 성품과 리더십
  - 청렴하고 성실했으며 병사들과의 신뢰, 병참·정비에 대한 철저한 관리로 유명하다.
  - 전시에도 일기(Nanjung Ilgi, 난중일기)를 남겨 당시의 전략·심정·환경을 자세히 기록했다. 난중일기는 역사적·문학적 가치가 큰 자료다.

- 최후
  - 1598년 노량해전에서 전사했다.

In [75]:
# 모델에 응답 스키마를 추가.
model_with_output = model.with_structured_output(OutputSchema)

res2 = model_with_output.invoke("이순신 장군에 대해 설명해줘.")

In [79]:
print(type(res2))
print(res2)
print(res2.is_real_person)
print(res2.description)

<class '__main__.OutputSchema'>
is_real_person=True description='이순신(李舜臣, 1545–1598)은 조선 중기(조선 선조 시대)의 해군 장군이자 군인으로, 임진왜란(1592–1598) 기간에 일본의 침입을 맞아 조선 수군을 지휘하며 결정적인 전과를 올린 한국의 대표적 국난 영웅이다. 그는 무과에 합격하여 군 경력을 시작했으며, 수군을 재편·강화하고 전술을 혁신하여 한산도 대첩, 명량 해전(13척으로 300여 척의 적을 격파) 등에서 승리했다. 거북선(거북선의 실제 설계·운용에 관한 세부 논쟁은 있으나 전통적으로 이순신이 관련된 것으로 알려짐) 등 판옥선 계열의 함정을 효과적으로 운용했고, 기동성과 지형을 이용한 전술로 해전에서 뛰어난 성과를 냈다. 그는 전쟁 중의 기록인 난중일기를 남겨 당시의 전황과 심경을 전해주며, 1598년 노량 해전에서 전사했다. 사후 충무공(忠武公)이라는 시호를 받았고, 오늘날 한국에서는 국민적 영웅으로 기념되며 그의 전략·리더십과 희생정신이 널리 존경받고 있다.'
True
이순신(李舜臣, 1545–1598)은 조선 중기(조선 선조 시대)의 해군 장군이자 군인으로, 임진왜란(1592–1598) 기간에 일본의 침입을 맞아 조선 수군을 지휘하며 결정적인 전과를 올린 한국의 대표적 국난 영웅이다. 그는 무과에 합격하여 군 경력을 시작했으며, 수군을 재편·강화하고 전술을 혁신하여 한산도 대첩, 명량 해전(13척으로 300여 척의 적을 격파) 등에서 승리했다. 거북선(거북선의 실제 설계·운용에 관한 세부 논쟁은 있으나 전통적으로 이순신이 관련된 것으로 알려짐) 등 판옥선 계열의 함정을 효과적으로 운용했고, 기동성과 지형을 이용한 전술로 해전에서 뛰어난 성과를 냈다. 그는 전쟁 중의 기록인 난중일기를 남겨 당시의 전황과 심경을 전해주며, 1598년 노량 해전에서 전사했다. 사후 충무공(忠武公)이라는 시호를 받았고, 오늘날 한국에서는 국민적 영웅으로 기념되며 그의 전략·리더십과 희생정신이 널리 

In [80]:
res3 = model_with_output.invoke("홍길동에 대해 설명해줘.")

In [81]:
res3

OutputSchema(is_real_person=False, description='홍길동은 조선 후기 소설 『홍길동전』에 등장하는 가공의 인물로, 전통적으로 허균(許筠)이 지은 작품으로 알려져 있다. 적서 차별로 인해 관직 진출에서 배제된 서얼 출신으로 설정되어 부당한 신분제와 부패한 관리들을 상대로 의적(義賊) 활동을 벌이며 약자에게 재산을 나눠 준다. 초인적 능력이 묘사되기도 하며, 마지막에는 나라를 떠나 외국에서 왕이 된다는 내용으로 끝나기도 한다. 이 작품은 신분제 비판과 사회정의를 다룬 대표적 한글소설로, 한국 문학과 대중문화에서 중요한 상징적 인물로 널리 알려져 있다.')

In [82]:
model.invoke("홍길동에 대해 설명해줘.")

AIMessage(content='홍길동은 한국 고전소설 「홍길동전」의 주인공으로, 사회제도와 부패한 양반층을 비판하는 상징적인 의적(義賊) 인물입니다.\n\n주요 내용과 특징\n- 작품: 「홍길동전」은 조선 후기(대체로 16–17세기경) 한글 소설로 평가되며, 한글 소설의 대표적·초기 작품 가운데 하나로 꼽힙니다. 작품의 실제 저자는 확실하지 않지만 허균(許筠, 1569–1618)이 지은 것으로 널리 알려져 있고, 학계에서는 저자 및 창작 연대에 관해 다양한 견해가 있습니다.\n- 줄거리 요약: 홍길동은 양반의 서자로 태어나 신분 때문에 억압을 받지만, 특별한 재능과 도술 같은 능력을 지녔습니다. 그는 부패한 관리와 불공정한 사회를 비판하며 도적단을 이끌고 양반과 권력자들의 재물을 훔쳐 가난한 사람들에게 나눠줍니다(로빈 후드 유형). 이후 조선을 떠나 이국(외국)에서 군공을 세우고 새로운 나라를 세우거나 다스리게 된다는 내용으로 마무리됩니다.\n- 주제와 의의: 신분제의 부당성, 사회 정의, 개인의 자유와 능력의 문제를 다루며 신분제 사회에 대한 비판과 인간 평등 의식을 담고 있다는 점에서 문학·사상사적으로 중요한 작품입니다.\n\n문화적 영향\n- 홍길동은 한국에서 전형적인 의적·영웅 인물로 자리잡아, 소설 외에도 만화·드라마·영화·뮤지컬 등으로 여러 차례 재창조되었습니다.\n- 또한 한국에서는 예시 이름이나 가명으로 ‘홍길동’이 자주 쓰입니다(예: 샘플 서식의 이름).\n\n더 알고 싶으시면 줄거리의 세부 전개(장면별 요약), 작품 원문 일부, 허균과의 관계에 관한 학술적 논쟁, 현대적 해석이나 주요 영상·문학적 각색 목록 등 중 어느 쪽을 원하시는지 알려주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1407, 'prompt_tokens': 15, 'total_tokens': 1422, 'completion_tokens_de

# Streaming 방식 응답 처리

- Streaming 방식 응답 처리란, LLM이 텍스트를 모두 생성할 때까지 기다리지 않고, 생성되는 즉시 **부분적인 결과**를 실시간으로 전달받아 처리하는 방식을 의미한다. 이는 사용자가 응답을 더 빠르게 인지할 수 있게 해 주며, 특히 대화형 서비스, 실시간 UI 출력, 긴 문서 생성과 같은 상황에서 매우 유용하게 활용된다.

- `invoke()` 요청으로 받는 응답은 **비 스트리밍 방식**으로 모든 응답 텍스트 생성이 완료된 이후 그 결과를 한 번에 반환하는 구조이다. 반면 Streaming 방식은 **토큰(token) 단위 또는 여러 토큰이 묶인 청크(chunk) 단위**로 연속적인 데이터 스트림을 전송한다는 점에서 큰 차이가 있다. 즉, Streaming은 마치 사람이 타이핑을 치듯이 응답을 실시간으로 “흘려보내는” 방식이라고 이해할 수 있다.

- `모델.invoke(input, config)` → 응답 데이터
    - 모델이 전체 응답을 모두 생성한 뒤, 최종 결과를 한 번에 반환하는 방식이다.
    - 배치 처리나 후처리가 중요한 경우에 적합하다.
- `모델.stream(input, config)` → Iterator
    - 모델이 토큰을 생성하는 즉시, 순차적으로 결과를 제공하는 Iterator 형태로 반환한다.
    - 실시간 출력, 대화형 인터페이스, 웹 스트리밍 등에 특히 적합하다.

In [84]:
model = ChatOpenAI(model="gpt-5-mini") #, streaming=True)

iterator = model.stream("서울의 유명한 관광지를 소개해줘. 추천 이유도 알려줘.")
print(type(iterator))
for token in iterator:
    print(token)

<class 'generator'>
content='' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content='아' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content='래' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content='는' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content=' 서울' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content='에서' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content=' 유명' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-ca7b-7162-8a33-1cab9d0785a0'
content='한' additional_kwargs={} response_metadata={'model_provide

In [85]:
for token in model.stream("서울의 유명한 관광지를 소개해줘. 추천 이유도 알려줘."):
    print(token.content, end="")

좋아요 — 서울의 대표 관광지들을 간단히 소개하고 각 장소를 추천하는 이유를 함께 적어드릴게요.

- 경복궁  
  설명: 조선 시대의 대표 궁궐이자 고궁 중 가장 규모가 큰 곳.  
  추천 이유: 왕실 건축과 궁중 문화를 직접 느낄 수 있고, 광화문·국립민속박물관 등과 한데 있어 역사 탐방 코스로 탁월합니다.  
  팁: 수문장 교대식(시간대 있음)과 한복 착용 시 무료 입장 혜택을 확인해보세요.

- 창덕궁 & 후원(비원)  
  설명: 조선의 궁궐 중 자연 경관과 조화가 뛰어난 곳으로, 후원(비밀의 정원)이 유명합니다.  
  추천 이유: 궁궐 건축과 정원 디자인의 조화를 감상할 수 있고, 후원은 특히 계절별 경관이 아름답습니다.  
  팁: 후원은 예약제 투어가 있으니 사전 확인 권장.

- 북촌한옥마을  
  설명: 전통 한옥들이 모여 있는 동네로 골목 산책에 좋습니다.  
  추천 이유: 전통 가옥과 현대 도시가 어우러진 풍경을 사진으로 남기기 좋고, 소규모 카페·공방도 많습니다.

- 인사동  
  설명: 한국 전통 공예품, 골동품, 찻집이 모여 있는 문화 거리.  
  추천 이유: 기념품 쇼핑과 전통 차·음식을 체험하기 좋고 전통미가 살아있는 거리 풍경을 즐길 수 있습니다.

- 명동  
  설명: 서울의 대표 쇼핑·패션 거리로 화장품과 스트리트 푸드가 풍성합니다.  
  추천 이유: 쇼핑과 길거리 음식, 외국인 관광객을 위한 편의시설이 잘 갖춰져 있어 초행자에게 편리합니다.

- 남대문·광장시장  
  설명: 남대문은 전통 재래시장, 광장시장은 먹거리·전통 물품이 많은 시장가.  
  추천 이유: 한국식 길거리 음식과 합리적인 쇼핑을 즐기기 좋고 현지 분위기를 느낄 수 있습니다.

- 동대문디자인플라자(DDP) & 동대문 쇼핑타운  
  설명: 미래지향적 건축과 야간에 열리는 패션·디자인 마켓으로 유명합니다.  
  추천 이유: 건축 사진 명소이자 밤에도 활기찬 패션 쇼핑을 즐기기에 좋습니다.

- N서울타워(남산)  
  설명: 남산 정상에